# TensorFlow Recommenders: Matrix Factorization Model

TensorFlow Recommenders (TFRS)는 TensorFlow 기반의 추천 시스템 라이브러리로, 복잡한 추천 시스템을 쉽게 구축하고 실험할 수 있도록 도와줍니다. TFRS는 모듈화된 구성 요소, 다양한 손실 함수와 평가 지표, 데이터 전처리 도구 등을 제공하여 추천 시스템 개발을 단순화합니다. Matrix Factorization 모델은 TFRS를 사용하여 구현할 수 있는 대표적인 모델로, 사용자와 아이템 임베딩을 통해 추천을 수행합니다.

https://www.tensorflow.org/recommenders/examples/quickstart

TFRS와 함께 [MovieLens 100K 데이터 세트](https://grouplens.org/datasets/movielens/100k/)를 사용하여 간단한 행렬 분해(matrix factorization) 모델을 구축합니다. 이 모델을 사용하여 특정 사용자에게 영화를 추천할 수 있습니다.

In [1]:
# TFRS와 호환되는 이전 version 설치
# 꼭 다시 시작한 직후 실행하세요!
!pip install --upgrade --force-reinstall -q \
  tensorflow==2.15.0 \
  tensorflow-recommenders==0.7.3 \
  tensorflow-datasets==4.9.4 \
  protobuf==3.20.* \
  ml-dtypes==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [1]:
# 타입 힌트를 위한 라이브러리 임포트
from typing import Dict, Text

import numpy as np
import tensorflow as tf

# 다양한 데이터셋을 쉽게 로드할 수 있는 라이브러리
import tensorflow_datasets as tfds

# 추천 시스템을 구축하기 위한 TensorFlow 라이브러리
import tensorflow_recommenders as tfrs

# 데이터를 보기 좋게 출력하기 위한 라이브러리
import pprint

In [19]:
# Version 확인
print("TF:", tf.__version__)
print("TFRS:", tfrs.__version__)
print("TFDS:", tfds.__version__)

TF: 2.15.0
TFRS: v0.7.3
TFDS: 4.9.4


### Read the data

- 'MovieLens 100K Ratings' 데이터셋은 영화 추천 시스템의 연구 및 개발에 널리 사용되며, 총 100,000개의 영화 평점 데이터를 포함.
    - 사용자 수: 943명
    - 영화 수: 1,682편
    - 평점 수: 100,000개
    - 평점 범위: 1점부터 5점까지의 정수

In [3]:
# 영화 평점 데이터를 불러옵니다.
ratings = tfds.load('movielens/100k-ratings', split="train")

# 모든 사용 가능한 영화의 특성 데이터를 불러옵니다.
movies = tfds.load('movielens/100k-movies', split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteVVVK24/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [4]:
# ratings 데이터셋에서 첫 번째 요소를 가져와 출력
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

# 줄바꿈을 위해 빈 줄 출력
print()

# movies 데이터셋에서 첫 번째 요소를 가져와 출력
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [5]:
# 'ratings' 데이터셋에서  'movie_title'과 'user_id' 필드만을 선택
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],  # 영화 제목
    "user_id": x["user_id"]           # 사용자 ID
})

# 'movies' 데이터셋에서 'movie_title' 필드만을 선택
movies = movies.map(lambda x: x["movie_title"])  # 영화 제목

In [6]:
# 필요한 필드만 선택 되었는지 확인
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

print()

for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'user_id': b'138'}

b'You So Crazy (1994)'


embedding layer를 위해 `사용자 ID`와 `영화 제목`을 정수 인덱스로 변환하는 vocabulary를 구축합니다.  

- `StringLookup` 레이어는 텍스트 데이터를 처리할 때 유용하게 사용됩니다. 이 경우, 사용자 ID와 영화 제목을 각각의 정수 인덱스로 매핑하여 모델이 이해할 수 있는 형태로 변환하는 역할을 합니다.  
-`adapt` 메소드는 주어진 데이터셋을 기반으로 어휘 사전을 자동으로 구성합니다. 이 과정에서 데이터셋의 모든 고유한 문자열 값을 스캔하여 인덱스를 할당합니다.  
- `mask_token=None` 옵션은 입력 데이터 중 어떤 값도 마스킹(무시)하지 않고 모두 처리하겠다는 것을 의미합니다. 일부 경우에는 특정 값을 무시하고 싶을 때 mask_token을 다르게 설정할 수 있습니다.  

이러한 어휘 사전을 사용하면 모델이 문자열 형태의 데이터를 쉽게 처리할 수 있으며, 추후에 이 데이터를 모델의 입력으로 사용할 때 일관된 방식으로 인코딩할 수 있습니다.

In [7]:
%%time

# 사용자 ID를 위한 어휘 사전 생성. mask_token=None은 어떤 토큰도 마스킹하지 않겠다는 의미입니다.
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)

# ratings 데이터에서 사용자 ID를 추출하여 어휘 사전에 적용(adapt)하여 인덱싱합니다.
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

# 영화 제목을 위한 어휘 사전 생성. 여기서도 mask_token=None으로 설정합니다.
movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)

# movies 데이터셋을 사용하여 영화 제목에 대한 어휘 사전을 구성합니다.
movie_titles_vocabulary.adapt(movies)

Cause: could not parse the source code of <function <lambda> at 0x7acd183c8860>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: could not parse the source code of <function <lambda> at 0x7acd183c8860>: no matching AST found among candidates:

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
CPU times: user 1min 49s, sys: 7.44 s, total: 1min 56s
Wall time: 2min 23s


In [8]:
# 사용자 id list
user_ids_vocabulary.get_vocabulary()[:10]

['[UNK]', '405', '655', '13', '450', '276', '416', '537', '303', '234']

In [9]:
# 영화 title list
movie_titles_vocabulary.get_vocabulary()[:10]

['[UNK]',
 "Ulee's Gold (1997)",
 'That Darn Cat! (1997)',
 'Substance of Fire, The (1996)',
 'Sliding Doors (1998)',
 'Nightwatch (1997)',
 'Money Talks (1997)',
 'Kull the Conqueror (1997)',
 'Ice Storm, The (1997)',
 'Hurricane Streets (1998)']

In [10]:
# 사용자 ID 3개를 선택하여 TensorFlow constant로 생성합니다.
data = tf.constant(['405', '655', '450'])

# 사용자 ID 어휘 사전을 사용하여 데이터에 포함된 사용자 ID를 정수 인덱스로 변환합니다.
# 이 변환된 인덱스는 모델의 입력으로 사용될 수 있으며, 각 사용자 ID가 어휘 사전에서 어떤 인덱스로 할당되었는지 확인할 수 있습니다.
user_ids_vocabulary(data)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 2, 4])>

In [11]:
# 영화 제목 데이터 3개를 선택하여 TensorFlow constant로  생성합니다.
data = tf.constant(["Ulee's Gold (1997)", 'Nightwatch (1997)', 'Ice Storm, The (1997)'])

# 영화 제목 어휘 사전을 사용하여 주어진 영화 제목들을 정수 인덱스로 변환합니다.
# 이 변환된 데이터는 모델의 입력으로 사용될 수 있으며, 각 영화 제목이 어휘 사전에서 어떤 인덱스로 할당되었는지 확인할 수 있습니다.
movie_titles_vocabulary(data)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([1, 5, 8])>

### model 정의

`tfrs.Model`을 상속하고 `compute_loss` 메서드를 구현하여 TFRS 모델을 정의할 수 있습니다.

In [12]:
class MovieLensModel(tfrs.Model):
  def __init__(self, user_model: tf.keras.Model,
                             movie_model: tf.keras.Model,
                             task: tfrs.tasks.Retrieval):
    super().__init__()

    # 사용자와 영화 표현(representation) 설정
    self.user_model = user_model  # 사용자 모델
    self.movie_model = movie_model  # 영화 모델
    # 검색(retrieval) 작업 설정
    self.task = task

  # 손실 함수(loss function) 계산을 위한 메소드
  # Dict[Text, tf.Tensor] --> 문자열 키와 Tensor값으로 구성된 dictionary
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # 사용자 ID로부터 사용자 임베딩 생성
    user_embeddings = self.user_model(features["user_id"])
    # 영화 제목으로부터 영화 임베딩 생성
    movie_embeddings = self.movie_model(features["movie_title"])

    # 생성된 사용자와 영화 임베딩을 이용하여 검색 작업을 수행하고, 손실을 반환
    return self.task(user_embeddings, movie_embeddings)

두 모델(사용자, 영화)과 retrieval task를 정의합니다.

tfrs.tasks.Retrieval: 이는 TFRS에서 제공하는 검색(retrieval) 작업을 설정하는 클래스입니다. 검색 작업은 사용자에게 관련 아이템을 추천하는 모델의 능력을 평가하고 최적화하는 데 중점을 둡니다.

    - tfrs.metrics.FactorizedTopK: 이 메트릭은 모델이 얼마나 잘 사용자에게 상위 K개의 관련 아이템을 추천하는지 평가합니다. 즉, 모델이 생성한 아이템 임베딩들 중 사용자의 실제 선호와 가장 유사한 아이템을 정확하게 찾아낼 수 있는지를 측정합니다.

    - movies.batch(128).map(movie_model): 이 부분은 FactorizedTopK 메트릭에 전달되는 아이템 후보 데이터셋을 준비하는 과정입니다. movies 데이터셋을 배치 크기 128로 나누고, 각 배치에 대해 movie_model을 적용하여 영화 임베딩을 생성합니다. 이렇게 생성된 임베딩들은 메트릭 계산에 사용되는 아이템 후보군으로 활용됩니다.


In [13]:
# 사용자 모델 정의 - 사용자 임베딩 생성
# 이 모델은 사용자 ID를 입력으로 받아 임베딩 벡터를 출력합니다.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,  # 사용자 ID를 정수 인덱스로 매핑
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)  # 매핑된 인덱스를 64차원의 임베딩 벡터로 변환
])

# 영화 모델 정의 - 영화 임베딩 생성
# 이 모델은 영화의 제목을 입력으로 받아 임베딩 벡터를 출력합니다.
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,  # 영화 제목을 정수 인덱스로 매핑
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)  # 매핑된 인덱스를 64차원의 임베딩 벡터로 변환
])

# 추천 시스템의 목적 함수 정의
# 생성된 영화 임베딩을 사용하여, 모델이 주어진 사용자에 대해 얼마나 관련성 높은 영화를 상위 K개로 추천할 수 있는지 평가합니다.
# FactorizedTopK 메트릭을 통해 계산된 성능 지표를 기반으로 모델을 학습하고 최적화합니다.
# 목표는 이 메트릭의 값을 최대화하는 것입니다.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)  # 영화 데이터셋을 배치로 묶고 각 배치에 영화 모델을 적용하여 임베딩을 생성합니다.
  )
)

영화 제목에 대한 임베딩을 직접 가져올 수 있습니다

In [14]:
# 학습된 영화 모델을 사용하여 "Star Wars (1977)" 영화의 임베딩 벡터를 예측합니다.
# 이 함수는 모델에 문자열 데이터를 입력으로 제공하고, 모델은 이 입력에 대한 영화의 특징을 나타내는 임베딩 벡터를 출력합니다.
movie_embedding = movie_model.predict(["Star Wars (1977)"])

# 예측된 임베딩 벡터의 차원을 출력합니다.
print(movie_embedding.shape)

# 예측된 임베딩 벡터를 출력합니다.
# 이 벡터는 영화 "Star Wars (1977)"의 특징을 수치적으로 표현한 것이며, 추천 시스템 등에서 사용될 수 있습니다.
movie_embedding

1/1 [==============================] - 0s 76ms/step
(1, 64)


array([[ 0.04538612, -0.01235112,  0.04931204,  0.04244621,  0.0346704 ,
         0.04954517,  0.02964803, -0.01108139, -0.036282  ,  0.02918825,
         0.04141038, -0.01618688, -0.01882006,  0.03039844,  0.00227008,
         0.03961113, -0.04341096,  0.02016329,  0.04489019,  0.01366863,
        -0.01318244,  0.04130575, -0.03536433, -0.01619341,  0.0379743 ,
        -0.00494099, -0.01268066,  0.02148655, -0.03741776,  0.02484265,
         0.03140701, -0.01394818, -0.00309738, -0.02137229, -0.00608271,
         0.0215071 , -0.02612066,  0.01806345, -0.03248224, -0.03357788,
        -0.0401055 ,  0.01941264, -0.00683467,  0.02287915, -0.03969206,
         0.02375248, -0.00547745,  0.01579256, -0.03375387, -0.01436484,
         0.00079615,  0.00087436, -0.02471856, -0.04310608,  0.014413  ,
         0.04202267, -0.00749002, -0.03053814,  0.03303078, -0.03950683,
         0.00413573,  0.01045779,  0.04183919, -0.02288312]],
      dtype=float32)


### Fit and evaluate it.

`tf.recommenders.layers.factorized_top_k`는 TensorFlow Recommenders (TFRS) 라이브러리 내에 있는 레이어 모음으로, 추천 시스템에서 사용자에게 아이템을 추천할 때 상위 K개의 가장 관련성 높은 아이템을 찾아내는 기능을 제공합니다. 이 레이어들은 추천 모델의 평가 단계에서 주로 사용되며, 모델이 얼마나 잘 사용자의 선호도를 예측하는지를 측정하는 데 도움을 줍니다.

- Label - 사용자가 실제로 상호작용한 아이템의 식별자입니다. 즉, 각 사용자에 대해 그들이 평가하거나 시청한 영화의 ID가 레이블로 사용됩니다.  
- Prediction - 모델이 예측한 사용자와 아이템(영화) 간의 선호도 또는 관련성 점수. 구체적으로, 사용자 임베딩과 아이템 임베딩의 내적(dot product)을 통해 계산된 유사도 점수입니다. 이 점수는 해당 사용자에게 아이템을 추천할 때의 우선순위를 결정하는 데 사용됩니다.


In [15]:
# 사용자 모델과 영화 모델을 결합한 검색 모델을 생성합니다.
# TFRS (TensorFlow Recommenders) 라이브러리를 활용하여 추천 모델을 구현합니다.
model = MovieLensModel(user_model, movie_model, task)

# 모델을 컴파일합니다. 손실 함수를 지정하지 않으면, TFRS 모델은 task 객체에 정의된 내장 손실 함수를 사용합니다.
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# 모델을 3 에폭 동안 훈련합니다. 배치 크기는 4096으로 설정합니다.
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0048 - factorized_top_k/top_50_categorical_accuracy: 0.0451 - factorized_top_k/top_100_categorical_accuracy: 0.1018 - loss: 33090.3924 - regularization_loss: 0.0000e+00 - total_loss: 33090.3924
Epoch 2/3
25/25 [==============================] - 28s 1s/step - factorized_top_k/top_1_categorical_accuracy: 3.2000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0055 - factorized_top_k/top_10_categorical_accuracy: 0.0150 - factorized_top_k/top_50_categorical_accuracy: 0.1054 - factorized_top_k/top_100_categorical_accuracy: 0.2110 - loss: 31012.0556 - regularization_loss: 0.0000e+00 - total_loss: 31012.0556
Epoch 3/3
25/25 [==============================] - 28s 1s/step - factorized_top_k/top_1_categorical_accuracy: 6.0000e-04 - factorized_top_k/top_5_categorical_accu

### `factorized_top_k` 의 item 검색 방법:

1. **BruteForce**:  모든 아이템 후보에 대해 순차적으로 사용자 쿼리와의 유사성(또는 거리)을 계산하고, 가장 유사한 상위 K개의 아이템을 반환. 이 과정은 계산적으로 비효율적일 수 있지만, 작은 데이터셋에서는 실용적일 수 있습니다.  

    - index_from_dataset 메서드는 TFRS 라이브러리의 BruteForce 레이어에서 제공하는 메서드로, 데이터셋에서 항목을 로드하여 인덱싱하는 데 사용됩니다. 이 메서드는 특히 추천 시스템에서 사용되며, 학습된 모델을 사용하여 생성된 임베딩을 인덱스 구조에 추가하여, 추후 검색(쿼리) 작업 시 빠르고 효율적으로 유사 항목을 찾을 수 있게 해줍니다.

2. **ScaNN**: 대규모 데이터셋에서 더 효율적인 근사 최근접 이웃 검색을 수행하는 레이어입니다.

In [16]:
# 훈련된 모델을 사용하여 무작위 탐색(brute-force search)을 설정합니다.
# 이를 통해 사용자에게 영화 추천을 수행할 수 있습니다.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

# 영화 데이터셋을 사용하여 무작위 탐색 인덱스를 생성합니다.
# 여기서는 영화 제목을 입력으로 받아 영화 모델이 반환한 임베딩을 인덱스에 추가합니다.
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))  # 영화 데이터셋으로부터 인덱스 생성

In [17]:
# 특정 사용자에 대한 추천 받기
_, titles = index(np.array(["42"]))  # 사용자 ID 42에 대한 상위 추천 영화 타이틀 가져오기
titles.numpy()

array([[b'Rent-a-Kid (1995)', b'Just Cause (1995)',
        b'All Dogs Go to Heaven 2 (1996)', b'House Arrest (1996)',
        b'Land Before Time III: The Time of the Great Giving (1995) (V)',
        b'Halloween: The Curse of Michael Myers (1995)', b'Andre (1994)',
        b'Panther (1995)', b'First Knight (1995)', b'Jack (1996)']],
      dtype=object)

In [18]:
# 특정 사용자에 대한 추천 받기
_, titles = index(np.array(["42"]))  # 사용자 ID 42에 대한 상위 추천 영화 타이틀 가져오기

# 상위 3개 추천 영화 타이틀을 문자열 리스트로 변환하여 출력
top_3_titles = [title.decode('utf-8') for title in titles[0, :3].numpy()]

print(f"Top 3 recommendations for user 42: ")
for title in top_3_titles:
    print(title)

Top 3 recommendations for user 42: 
Rent-a-Kid (1995)
Just Cause (1995)
All Dogs Go to Heaven 2 (1996)
